In [1]:
import argparse
import os
import joblib
from sklearn import datasets
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [2]:
def train_model(train_X, train_y):
    model = svm.SVC(gamma='scale')
    model.fit(train_X, train_y)
    return model

    
def eval_model(model, test_X, test_y):
    predictions = model.predict(test_X)
    print('mean_absolute_error={}'.format(mean_absolute_error(predictions, test_y)))

    
def save_model(model, model_file):
    joblib.dump(model, model_file)
    print('Saved model to {}'.format(model_file))

In [3]:
class ModelServe(object):
   
    def __init__(self):
        self.model_file = 'trained_model.dat'
        self.trained_model = None

        
    def train(self):
        iris = datasets.load_iris()
        X, y = iris.data, iris.target
        train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2)
        model = train_model(train_X, train_y)

        eval_model(model, test_X, test_y)
        save_model(model, self.model_file)

        
    def predict(self, X, feature_names):
        if not self.trained_model:
            self.trained_model = joblib.load(self.model_file)
        predictions = self.trained_model.predict(X)
        return predictions

In [4]:
ModelServe().train()

mean_absolute_error=0.0
Saved model to trained_model.dat


In [5]:
from kubeflow import fairing
from kubeflow.fairing.kubernetes import utils as k8s_utils
 

CONTAINER_REGISTRY = 'kangwoo'

In [6]:
fairing.config.set_preprocessor("function", function_obj=ModelServe,
                                input_files=["trained_model.dat"])
fairing.config.set_builder('append', registry=CONTAINER_REGISTRY, image_name="fairing-function-append-serving", base_image='kangwoo/sklearn-seldon:0.0.1')
fairing.config.set_deployer('serving', serving_class="ModelServe")
_, _, deployer = fairing.config.run()

[I 200501 03:28:39 config:134] Using preprocessor: <kubeflow.fairing.preprocessors.function.FunctionPreProcessor object at 0x7f1ec80edb00>
[I 200501 03:28:39 config:136] Using builder: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7f1ee88830f0>
[I 200501 03:28:39 config:138] Using deployer: <kubeflow.fairing.deployers.serving.serving.Serving object at 0x7f1f100f45f8>
[W 200501 03:28:39 append:50] Building image using Append builder...
[I 200501 03:28:39 base:107] Creating docker context: /tmp/fairing_context_z0x0f5uj
[W 200501 03:28:39 base:94] /home/jovyan/.local/lib/python3.6/site-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
[I 200501 03:28:39 docker_creds_:234] Loading Docker credentials for repository 'kangwoo/sklearn-seldon:0.0.1'
[W 200501 03:28:41 append:54] Image successfully built in 2.501330686998699s.
[W 200501 03:28:41 append:94] Pushing image kangwoo/fairing-function-append-serving:2D8363EC...
[I 200501 03:28:41 

In [7]:
!curl http://fairing-service-s49mg.admin.svc.cluster.local:5000/predict -H "Content-Type: application/x-www-form-urlencoded" -d 'json={"data": {"ndarray": [[6.8,  2.8,  4.8,  1.4], [6.0,  3.4,  4.5,  1.6]]}}'

{"data":{"names":[],"ndarray":[1,1]},"meta":{}}


In [8]:
job_id = deployer.job_id
print(deployer.job_id)

de101e48-8b5b-11ea-ace7-ce20adfccf14


In [9]:
!kubectl get deployment -l fairing-deployer=serving,fairing-id={job_id}

NAME                     READY   UP-TO-DATE   AVAILABLE   AGE
fairing-deployer-wjrtn   1/1     1            1           9s


In [10]:
!kubectl get service -l fairing-deployer=serving,fairing-id={job_id}

NAME                    TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)    AGE
fairing-service-s49mg   ClusterIP   10.102.122.61   <none>        5000/TCP   11s


In [11]:
deployer.delete()

[I 200501 03:28:59 serving:124] Deleted service: admin/fairing-service-s49mg
[I 200501 03:28:59 serving:136] Deleted deployment: admin/fairing-deployer-wjrtn
